In [80]:
import duckdb
import pandas as pd

In [81]:
conn = duckdb.connect("../db/synpuf.duckdb")

In [16]:
conn.execute("""
CREATE OR REPLACE TABLE beneficiary_all AS
WITH stacked AS (
    SELECT *, 2008 AS year FROM beneficiary_2008
    UNION ALL
    SELECT *, 2009 AS year FROM beneficiary_2009
    UNION ALL
    SELECT *, 2010 AS year FROM beneficiary_2010
),
ranked AS (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY DESYNPUF_ID ORDER BY year DESC) AS rn
    FROM stacked
)
SELECT *
FROM ranked
WHERE rn = 1;
""")

conn.execute("""
CREATE OR REPLACE TABLE chronic_all AS
WITH stacked AS (
    SELECT *, 2008 AS year FROM chronic_2008
    UNION ALL
    SELECT *, 2009 AS year FROM chronic_2009
    UNION ALL
    SELECT *, 2010 AS year FROM chronic_2010
),
ranked AS (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY DESYNPUF_ID ORDER BY year DESC) AS rn
    FROM stacked
)
SELECT *
FROM ranked
WHERE rn = 1;
""")

In [20]:
conn.execute("""
CREATE OR REPLACE TABLE inpatient_agg AS
SELECT 
    DESYNPUF_ID,

    -- Claim-level metrics
    COUNT(DISTINCT CLM_ID) AS inpatient_num_claims,
    SUM(CLM_PMT_AMT) AS inpatient_total_payment,
    SUM(NCH_PRMRY_PYR_CLM_PD_AMT) AS inpatient_primary_payer_amt,
    SUM(NCH_BENE_IP_DDCTBL_AMT) AS inpatient_total_deductible,
    SUM(NCH_BENE_PTA_COINSRNC_LBLTY_AM) AS inpatient_total_coinsurance,
    SUM(NCH_BENE_BLOOD_DDCTBL_LBLTY_AM) AS inpatient_total_blood_deductible,
    SUM(CLM_UTLZTN_DAY_CNT) AS inpatient_total_util_days,

    -- Unique providers / physicians
    COUNT(DISTINCT PRVDR_NUM) AS num_unique_providers,
    COUNT(DISTINCT AT_PHYSN_NPI) AS num_unique_attending_physicians,
    COUNT(DISTINCT OP_PHYSN_NPI) AS num_unique_operating_physicians,
    COUNT(DISTINCT OT_PHYSN_NPI) AS num_unique_other_physicians,

    -- DRG diversity
    COUNT(DISTINCT CLM_DRG_CD) AS num_unique_drg,

    -- Diagnosis codes (1–10)
    COUNT(DISTINCT ICD9_DGNS_CD_1) 
      + COUNT(DISTINCT ICD9_DGNS_CD_2)
      + COUNT(DISTINCT ICD9_DGNS_CD_3)
      + COUNT(DISTINCT ICD9_DGNS_CD_4)
      + COUNT(DISTINCT ICD9_DGNS_CD_5)
      + COUNT(DISTINCT ICD9_DGNS_CD_6)
      + COUNT(DISTINCT ICD9_DGNS_CD_7)
      + COUNT(DISTINCT ICD9_DGNS_CD_8)
      + COUNT(DISTINCT ICD9_DGNS_CD_9)
      + COUNT(DISTINCT ICD9_DGNS_CD_10) AS num_unique_icd9_diagnosis,

    -- Procedure codes (1–6)
    COUNT(DISTINCT ICD9_PRCDR_CD_1) 
      + COUNT(DISTINCT ICD9_PRCDR_CD_2)
      + COUNT(DISTINCT ICD9_PRCDR_CD_3)
      + COUNT(DISTINCT ICD9_PRCDR_CD_4)
      + COUNT(DISTINCT ICD9_PRCDR_CD_5)
      + COUNT(DISTINCT ICD9_PRCDR_CD_6) AS num_unique_icd9_procedure,

    -- HCPCS (1–45)
    COUNT(DISTINCT HCPCS_CD_1)  + COUNT(DISTINCT HCPCS_CD_2)  + COUNT(DISTINCT HCPCS_CD_3)  +
    COUNT(DISTINCT HCPCS_CD_4)  + COUNT(DISTINCT HCPCS_CD_5)  + COUNT(DISTINCT HCPCS_CD_6)  +
    COUNT(DISTINCT HCPCS_CD_7)  + COUNT(DISTINCT HCPCS_CD_8)  + COUNT(DISTINCT HCPCS_CD_9)  +
    COUNT(DISTINCT HCPCS_CD_10) + COUNT(DISTINCT HCPCS_CD_11) + COUNT(DISTINCT HCPCS_CD_12) +
    COUNT(DISTINCT HCPCS_CD_13) + COUNT(DISTINCT HCPCS_CD_14) + COUNT(DISTINCT HCPCS_CD_15) +
    COUNT(DISTINCT HCPCS_CD_16) + COUNT(DISTINCT HCPCS_CD_17) + COUNT(DISTINCT HCPCS_CD_18) +
    COUNT(DISTINCT HCPCS_CD_19) + COUNT(DISTINCT HCPCS_CD_20) + COUNT(DISTINCT HCPCS_CD_21) +
    COUNT(DISTINCT HCPCS_CD_22) + COUNT(DISTINCT HCPCS_CD_23) + COUNT(DISTINCT HCPCS_CD_24) +
    COUNT(DISTINCT HCPCS_CD_25) + COUNT(DISTINCT HCPCS_CD_26) + COUNT(DISTINCT HCPCS_CD_27) +
    COUNT(DISTINCT HCPCS_CD_28) + COUNT(DISTINCT HCPCS_CD_29) + COUNT(DISTINCT HCPCS_CD_30) +
    COUNT(DISTINCT HCPCS_CD_31) + COUNT(DISTINCT HCPCS_CD_32) + COUNT(DISTINCT HCPCS_CD_33) +
    COUNT(DISTINCT HCPCS_CD_34) + COUNT(DISTINCT HCPCS_CD_35) + COUNT(DISTINCT HCPCS_CD_36) +
    COUNT(DISTINCT HCPCS_CD_37) + COUNT(DISTINCT HCPCS_CD_38) + COUNT(DISTINCT HCPCS_CD_39) +
    COUNT(DISTINCT HCPCS_CD_40) + COUNT(DISTINCT HCPCS_CD_41) + COUNT(DISTINCT HCPCS_CD_42) +
    COUNT(DISTINCT HCPCS_CD_43) + COUNT(DISTINCT HCPCS_CD_44) + COUNT(DISTINCT HCPCS_CD_45) AS num_unique_hcpcs

FROM inpatient
GROUP BY DESYNPUF_ID;

""")

In [24]:
conn.execute("""
CREATE OR REPLACE TABLE outpatient_agg AS
SELECT 
    DESYNPUF_ID,

    -- Claim-level metrics
    COUNT(DISTINCT CLM_ID) AS outpatient_num_claims,
    SUM(CLM_PMT_AMT) AS outpatient_total_payment,
    SUM(NCH_PRMRY_PYR_CLM_PD_AMT) AS outpatient_primary_payer_amt,
    SUM(NCH_BENE_PTB_DDCTBL_AMT) AS outpatient_total_deductible,
    SUM(NCH_BENE_PTB_COINSRNC_AMT) AS outpatient_total_coinsurance,
    SUM(NCH_BENE_BLOOD_DDCTBL_LBLTY_AM) AS outpatient_total_blood_deductible,

    -- Unique providers / physicians
    COUNT(DISTINCT PRVDR_NUM) AS num_unique_providers,
    COUNT(DISTINCT AT_PHYSN_NPI) AS num_unique_attending_physicians,
    COUNT(DISTINCT OP_PHYSN_NPI) AS num_unique_operating_physicians,
    COUNT(DISTINCT OT_PHYSN_NPI) AS num_unique_other_physicians,

    -- Diagnosis codes (1–10)
    COUNT(DISTINCT ICD9_DGNS_CD_1) 
      + COUNT(DISTINCT ICD9_DGNS_CD_2)
      + COUNT(DISTINCT ICD9_DGNS_CD_3)
      + COUNT(DISTINCT ICD9_DGNS_CD_4)
      + COUNT(DISTINCT ICD9_DGNS_CD_5)
      + COUNT(DISTINCT ICD9_DGNS_CD_6)
      + COUNT(DISTINCT ICD9_DGNS_CD_7)
      + COUNT(DISTINCT ICD9_DGNS_CD_8)
      + COUNT(DISTINCT ICD9_DGNS_CD_9)
      + COUNT(DISTINCT ICD9_DGNS_CD_10) AS num_unique_icd9_diagnosis,

    -- Admission diagnosis (single field, can treat as one more slot)
    COUNT(DISTINCT ADMTNG_ICD9_DGNS_CD) AS num_unique_admitting_diagnosis,

    -- Procedure codes (1–6)
    COUNT(DISTINCT ICD9_PRCDR_CD_1) 
      + COUNT(DISTINCT ICD9_PRCDR_CD_2)
      + COUNT(DISTINCT ICD9_PRCDR_CD_3)
      + COUNT(DISTINCT ICD9_PRCDR_CD_4)
      + COUNT(DISTINCT ICD9_PRCDR_CD_5)
      + COUNT(DISTINCT ICD9_PRCDR_CD_6) AS num_unique_icd9_procedure,

    -- HCPCS (1–45)
    COUNT(DISTINCT HCPCS_CD_1)  + COUNT(DISTINCT HCPCS_CD_2)  + COUNT(DISTINCT HCPCS_CD_3)  +
    COUNT(DISTINCT HCPCS_CD_4)  + COUNT(DISTINCT HCPCS_CD_5)  + COUNT(DISTINCT HCPCS_CD_6)  +
    COUNT(DISTINCT HCPCS_CD_7)  + COUNT(DISTINCT HCPCS_CD_8)  + COUNT(DISTINCT HCPCS_CD_9)  +
    COUNT(DISTINCT HCPCS_CD_10) + COUNT(DISTINCT HCPCS_CD_11) + COUNT(DISTINCT HCPCS_CD_12) +
    COUNT(DISTINCT HCPCS_CD_13) + COUNT(DISTINCT HCPCS_CD_14) + COUNT(DISTINCT HCPCS_CD_15) +
    COUNT(DISTINCT HCPCS_CD_16) + COUNT(DISTINCT HCPCS_CD_17) + COUNT(DISTINCT HCPCS_CD_18) +
    COUNT(DISTINCT HCPCS_CD_19) + COUNT(DISTINCT HCPCS_CD_20) + COUNT(DISTINCT HCPCS_CD_21) +
    COUNT(DISTINCT HCPCS_CD_22) + COUNT(DISTINCT HCPCS_CD_23) + COUNT(DISTINCT HCPCS_CD_24) +
    COUNT(DISTINCT HCPCS_CD_25) + COUNT(DISTINCT HCPCS_CD_26) + COUNT(DISTINCT HCPCS_CD_27) +
    COUNT(DISTINCT HCPCS_CD_28) + COUNT(DISTINCT HCPCS_CD_29) + COUNT(DISTINCT HCPCS_CD_30) +
    COUNT(DISTINCT HCPCS_CD_31) + COUNT(DISTINCT HCPCS_CD_32) + COUNT(DISTINCT HCPCS_CD_33) +
    COUNT(DISTINCT HCPCS_CD_34) + COUNT(DISTINCT HCPCS_CD_35) + COUNT(DISTINCT HCPCS_CD_36) +
    COUNT(DISTINCT HCPCS_CD_37) + COUNT(DISTINCT HCPCS_CD_38) + COUNT(DISTINCT HCPCS_CD_39) +
    COUNT(DISTINCT HCPCS_CD_40) + COUNT(DISTINCT HCPCS_CD_41) + COUNT(DISTINCT HCPCS_CD_42) +
    COUNT(DISTINCT HCPCS_CD_43) + COUNT(DISTINCT HCPCS_CD_44) + COUNT(DISTINCT HCPCS_CD_45) AS num_unique_hcpcs

FROM outpatient
GROUP BY DESYNPUF_ID;

""")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [149]:
import duckdb
import pandas as pd

# Connect to DuckDB
con = duckdb.connect("../db/synpuf.duckdb")

# Inspect row count
row_count = con.execute("SELECT COUNT(*) FROM carrier").fetchone()[0]
print("Carrier total rows:", row_count)

# Diagnostic query
diag_query = """
SELECT
    COUNT(*) AS total_rows,
    COUNT(*) FILTER (WHERE HCPCS_CD_1 IS NOT NULL AND HCPCS_CD_1 <> '') AS non_empty_hcpcs,
    COUNT(*) FILTER (WHERE LINE_PRCSG_IND_CD_1 IS NOT NULL AND LINE_PRCSG_IND_CD_1 <> '') AS non_empty_service
FROM carrier;
"""
diag_result = con.execute(diag_query).fetchone()
print("Diagnostic - Total rows:", diag_result[0], 
      "| Non-empty HCPCS_CD_1:", diag_result[1], 
      "| Non-empty LINE_PRCSG_IND_CD_1:", diag_result[2])

# Main query: Flatten lines with UNION ALL
query = """
WITH lines AS (
    SELECT DESYNPUF_ID, CLM_ID,
           LINE_NCH_PMT_AMT_1 AS LINE_NCH_PMT_AMT,
           LINE_BENE_PTB_DDCTBL_AMT_1 AS LINE_BENE_PTB_DDCTBL_AMT,
           LINE_COINSRNC_AMT_1 AS LINE_BENE_COINSRNC_AMT,
           HCPCS_CD_1 AS HCPCS_CD,
           LINE_PRCSG_IND_CD_1 AS LINE_CMS_TYPE_SRVC_CD
    FROM carrier
    UNION ALL
    SELECT DESYNPUF_ID, CLM_ID,
           LINE_NCH_PMT_AMT_2, LINE_BENE_PTB_DDCTBL_AMT_2, LINE_COINSRNC_AMT_2,
           HCPCS_CD_2, LINE_PRCSG_IND_CD_2
    FROM carrier
    UNION ALL
    SELECT DESYNPUF_ID, CLM_ID,
           LINE_NCH_PMT_AMT_3, LINE_BENE_PTB_DDCTBL_AMT_3, LINE_COINSRNC_AMT_3,
           HCPCS_CD_3, LINE_PRCSG_IND_CD_3
    FROM carrier
    UNION ALL
    SELECT DESYNPUF_ID, CLM_ID,
           LINE_NCH_PMT_AMT_4, LINE_BENE_PTB_DDCTBL_AMT_4, LINE_COINSRNC_AMT_4,
           HCPCS_CD_4, LINE_PRCSG_IND_CD_4
    FROM carrier
    UNION ALL
    SELECT DESYNPUF_ID, CLM_ID,
           LINE_NCH_PMT_AMT_4, LINE_BENE_PTB_DDCTBL_AMT_4, LINE_COINSRNC_AMT_4,
           HCPCS_CD_4, LINE_PRCSG_IND_CD_4
    FROM carrier
    UNION ALL
    SELECT DESYNPUF_ID, CLM_ID,
           LINE_NCH_PMT_AMT_5, LINE_BENE_PTB_DDCTBL_AMT_5, LINE_COINSRNC_AMT_5,
           HCPCS_CD_5, LINE_PRCSG_IND_CD_5
    FROM carrier
    UNION ALL
    SELECT DESYNPUF_ID, CLM_ID,
           LINE_NCH_PMT_AMT_6, LINE_BENE_PTB_DDCTBL_AMT_6, LINE_COINSRNC_AMT_6,
           HCPCS_CD_6, LINE_PRCSG_IND_CD_6
    FROM carrier
    UNION ALL
    SELECT DESYNPUF_ID, CLM_ID,
           LINE_NCH_PMT_AMT_7, LINE_BENE_PTB_DDCTBL_AMT_7, LINE_COINSRNC_AMT_7,
           HCPCS_CD_7, LINE_PRCSG_IND_CD_7
    FROM carrier
    UNION ALL
    SELECT DESYNPUF_ID, CLM_ID,
           LINE_NCH_PMT_AMT_8, LINE_BENE_PTB_DDCTBL_AMT_8, LINE_COINSRNC_AMT_8,
           HCPCS_CD_8, LINE_PRCSG_IND_CD_8
    FROM carrier
    UNION ALL
    SELECT DESYNPUF_ID, CLM_ID,
           LINE_NCH_PMT_AMT_9, LINE_BENE_PTB_DDCTBL_AMT_9, LINE_COINSRNC_AMT_9,
           HCPCS_CD_9, LINE_PRCSG_IND_CD_9
    FROM carrier
    UNION ALL
    SELECT DESYNPUF_ID, CLM_ID,
           LINE_NCH_PMT_AMT_10, LINE_BENE_PTB_DDCTBL_AMT_10, LINE_COINSRNC_AMT_10,
           HCPCS_CD_10, LINE_PRCSG_IND_CD_10
    FROM carrier
    UNION ALL
    SELECT DESYNPUF_ID, CLM_ID,
           LINE_NCH_PMT_AMT_11, LINE_BENE_PTB_DDCTBL_AMT_11, LINE_COINSRNC_AMT_11,
           HCPCS_CD_11, LINE_PRCSG_IND_CD_11
    FROM carrier
    UNION ALL
    SELECT DESYNPUF_ID, CLM_ID,
           LINE_NCH_PMT_AMT_12, LINE_BENE_PTB_DDCTBL_AMT_12, LINE_COINSRNC_AMT_12,
           HCPCS_CD_12, LINE_PRCSG_IND_CD_12
    FROM carrier
    UNION ALL
    SELECT DESYNPUF_ID, CLM_ID,
           LINE_NCH_PMT_AMT_13, LINE_BENE_PTB_DDCTBL_AMT_13, LINE_COINSRNC_AMT_13,
           HCPCS_CD_13, LINE_PRCSG_IND_CD_13
    FROM carrier
)
SELECT
    DESYNPUF_ID,
    COUNT(DISTINCT CLM_ID) AS carrier_num_claims,
    COUNT(*) AS carrier_num_lines,
    SUM(COALESCE(LINE_NCH_PMT_AMT, 0)) AS carrier_total_payment,
    SUM(COALESCE(LINE_BENE_PTB_DDCTBL_AMT, 0)) AS carrier_total_deductible,
    SUM(COALESCE(LINE_BENE_COINSRNC_AMT, 0)) AS carrier_total_coinsurance,
    COUNT(DISTINCT HCPCS_CD) FILTER (WHERE HCPCS_CD IS NOT NULL AND HCPCS_CD <> '') AS carrier_num_unique_hcpcs,
    COUNT(DISTINCT LINE_CMS_TYPE_SRVC_CD) FILTER (WHERE LINE_CMS_TYPE_SRVC_CD IS NOT NULL AND LINE_CMS_TYPE_SRVC_CD <> '') AS carrier_num_unique_service_types
FROM lines
GROUP BY DESYNPUF_ID;
"""

# Execute query and convert to DataFrame
carrier_final = con.execute(query).df()

# Debug: Inspect the output
print("Sample of carrier_final:")
print(carrier_final[["DESYNPUF_ID", "carrier_num_claims", "carrier_num_lines", 
                     "carrier_total_payment", "carrier_num_unique_hcpcs", 
                     "carrier_num_unique_service_types"]].head(10))

print("Summary of carrier_num_unique_hcpcs:")
print(carrier_final["carrier_num_unique_hcpcs"].describe())

print("Summary of carrier_num_unique_service_types:")
print(carrier_final["carrier_num_unique_service_types"].describe())

# Save final results back to DuckDB
con.register("carrier_final_df", carrier_final)
con.execute("CREATE OR REPLACE TABLE carrier_agg AS SELECT * FROM carrier_final_df")

print("✅ carrier_agg table created with", len(carrier_final), "patients")


Carrier total rows: 4741335
Diagnostic - Total rows: 4741335 | Non-empty HCPCS_CD_1: 4687708 | Non-empty LINE_PRCSG_IND_CD_1: 4741335


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Sample of carrier_final:
        DESYNPUF_ID  carrier_num_claims  carrier_num_lines  \
0  0CB31BA13E4FDDC5                  57                798   
1  0CBEFC3C88B3554C                  25                350   
2  0CD17CA991EF33CE                  30                420   
3  0CD1D7724029F76E                  75               1050   
4  0D0187B51C47AF28                   3                 42   
5  0D069DBD6642A53A                  27                378   
6  0D120A7CBD3167E9                 123               1722   
7  0D1244358E149B38                   2                 28   
8  0D132673894DDC35                  16                224   
9  0D36CF9683948B61                  28                392   

   carrier_total_payment  carrier_num_unique_hcpcs  \
0                 4520.0                        52   
1                 1630.0                        34   
2                 1770.0                        28   
3                 8630.0                        82   
4                  240

In [157]:
import duckdb

# connect
con = duckdb.connect("../db/synpuf.duckdb")

# aggregate PDE
query = """
CREATE OR REPLACE TABLE pde_agg AS
SELECT
    DESYNPUF_ID,
    COUNT(DISTINCT PDE_ID) AS pde_num_prescriptions,
    COUNT(DISTINCT PROD_SRVC_ID) AS pde_num_unique_drugs,
    SUM(QTY_DSPNSD_NUM) AS pde_total_quantity,
    AVG(QTY_DSPNSD_NUM) AS pde_avg_quantity,
    SUM(DAYS_SUPLY_NUM) AS pde_total_days_supply,
    AVG(DAYS_SUPLY_NUM) AS pde_avg_days_supply,
    SUM(PTNT_PAY_AMT) AS pde_total_patient_payment,
    SUM(TOT_RX_CST_AMT) AS pde_total_drug_cost,
    AVG(TOT_RX_CST_AMT) AS pde_avg_drug_cost
FROM pde
GROUP BY DESYNPUF_ID
"""

con.execute(query)
print("✅ pde_agg table created")


✅ pde_agg table created


In [150]:
df = conn.execute("select * from carrier_agg limit 10000").df()

In [151]:
df

,DESYNPUF_ID,carrier_num_claims,carrier_num_lines,carrier_total_payment,carrier_total_deductible,carrier_total_coinsurance,carrier_num_unique_hcpcs,carrier_num_unique_service_types
0,0CB31BA13E4FDDC5,57,798,4520.0,130.0,1000.0,52,5
1,0CBEFC3C88B3554C,25,350,1630.0,330.0,570.0,34,4
2,0CD17CA991EF33CE,30,420,1770.0,0.0,390.0,28,5
3,0CD1D7724029F76E,75,1050,8630.0,470.0,1900.0,82,5
4,0D0187B51C47AF28,3,42,240.0,0.0,40.0,4,2
...,...,...,...,...,...,...,...,...
9995,2A8CC00DFE7BACC5,87,1218,11770.0,130.0,2790.0,114,7
9996,2AA6593AE08CE2DA,45,630,3520.0,120.0,720.0,60,5
9997,2AAE757BEFF1843A,57,798,6150.0,130.0,1390.0,84,5
9998,2AB62EAD13948089,2,28,60.0,0.0,10.0,3,1


In [ ]:
conn.execute("""
SELECT col, code, freq
FROM (
    SELECT 'HCPCS_CD_1' AS col, HCPCS_CD_1 AS code, COUNT(*) AS freq
    FROM carrier
    GROUP BY HCPCS_CD_1
    ORDER BY freq DESC
    LIMIT 10
)
UNION ALL
SELECT col, code, freq
FROM (
    SELECT 'HCPCS_CD_2' AS col, HCPCS_CD_2 AS code, COUNT(*) AS freq
    FROM carrier
    GROUP BY HCPCS_CD_2
    ORDER BY freq DESC
    LIMIT 10
)
UNION ALL
SELECT col, code, freq
FROM (
    SELECT 'HCPCS_CD_3' AS col, HCPCS_CD_3 AS code, COUNT(*) AS freq
    FROM carrier
    GROUP BY HCPCS_CD_3
    ORDER BY freq DESC
    LIMIT 10
);
""").fetch_df()

,col,code,freq
0,HCPCS_CD_1,99213,573229
1,HCPCS_CD_1,99214,413258
2,HCPCS_CD_1,36415,150496
3,HCPCS_CD_1,99232,112838
4,HCPCS_CD_1,99212,109174
5,HCPCS_CD_1,71010,96930
6,HCPCS_CD_1,93010,82680
7,HCPCS_CD_1,71020,72485
8,HCPCS_CD_1,98941,62007
9,HCPCS_CD_1,92014,60607


In [ ]:
query = """
CREATE OR REPLACE TABLE patient_master AS
SELECT 
    b.DESYNPUF_ID,

    -- Demographics
    b.BENE_SEX_IDENT_CD AS sex,
    b.BENE_BIRTH_DT AS birth_date,
    b.BENE_RACE_CD AS race,
    b.BENE_DEATH_DT AS death_date,

    -- Chronic conditions
    c.* EXCLUDE (DESYNPUF_ID),

    -- Inpatient aggregates
    i.* EXCLUDE (DESYNPUF_ID),

    -- Outpatient aggregates
    o.* EXCLUDE (DESYNPUF_ID),

    -- Carrier aggregates
    ca.* EXCLUDE (DESYNPUF_ID),

    -- Drug (PDE) aggregates
    p.* EXCLUDE (DESYNPUF_ID)

FROM beneficiary_all b
LEFT JOIN chronic_all    c  ON b.DESYNPUF_ID = c.DESYNPUF_ID
LEFT JOIN inpatient_agg  i  ON b.DESYNPUF_ID = i.DESYNPUF_ID
LEFT JOIN outpatient_agg o  ON 🎯 Candidate 10 Narrative Features

    inpatient_num_claims → hospital admissions

    inpatient_total_payment → cost burden of hospitalizations

    outpatient_num_claims → outpatient visits (frequency of care)

    carrier_num_lines → number of billed services (granular activity)

    carrier_num_unique_hcpcs → variety of procedures

    pde_num_prescriptions → volume of medication use

    pde_num_unique_drugs → complexity of medication regimen

    pde_total_drug_cost → medication cost burden

    chronic_count_2008 → baseline chronic disease burden

    total_spending → overall healthcare cost (catch-all narrative anchor)
b.DESYNPUF_ID = o.DESYNPUF_ID
LEFT JOIN carrier_agg    ca ON b.DESYNPUF_ID = ca.DESYNPUF_ID
LEFT JOIN pde_agg        p  ON b.DESYNPUF_ID = p.DESYNPUF_ID;
"""
conn.execute(query)

In [160]:
df = conn.execute("select * from patient_master").df()

In [161]:
df.count()

DESYNPUF_ID                          116352
sex                                  116352
birth_date                           116352
race                                 116352
death_date                             5461
chronic_count_2008                   116352
year                                 116352
rn                                   116352
inpatient_num_claims                  37780
inpatient_total_payment               37780
inpatient_primary_payer_amt           37780
inpatient_total_deductible            37780
inpatient_total_coinsurance           37780
inpatient_total_blood_deductible      37780
inpatient_total_util_days             37780
num_unique_providers                  37780
num_unique_attending_physicians       37780
num_unique_operating_physicians       37780
num_unique_other_physicians           37780
num_unique_drg                        37780
num_unique_icd9_diagnosis             37780
num_unique_icd9_procedure             37780
num_unique_hcpcs                

In [162]:
query = """
CREATE OR REPLACE TABLE patient_master_clean AS
SELECT
    DESYNPUF_ID,
    sex,
    birth_date,
    race,
    death_date,
    chronic_count_2008,
    year,
    rn,

    -- Inpatient (COALESCE to 0)
    COALESCE(inpatient_num_claims, 0)                 AS inpatient_num_claims,
    COALESCE(inpatient_total_payment, 0)              AS inpatient_total_payment,
    COALESCE(inpatient_primary_payer_amt, 0)          AS inpatient_primary_payer_amt,
    COALESCE(inpatient_total_deductible, 0)           AS inpatient_total_deductible,
    COALESCE(inpatient_total_coinsurance, 0)          AS inpatient_total_coinsurance,
    COALESCE(inpatient_total_blood_deductible, 0)     AS inpatient_total_blood_deductible,
    COALESCE(inpatient_total_util_days, 0)            AS inpatient_total_util_days,
    COALESCE(num_unique_providers, 0)                 AS num_unique_inpatient_providers,
    COALESCE(num_unique_attending_physicians, 0)      AS num_unique_inpatient_attending,
    COALESCE(num_unique_operating_physicians, 0)      AS num_unique_inpatient_operating,
    COALESCE(num_unique_other_physicians, 0)          AS num_unique_inpatient_other,
    COALESCE(num_unique_drg, 0)                       AS num_unique_drg,
    COALESCE(num_unique_icd9_diagnosis, 0)            AS num_unique_inpatient_dx,
    COALESCE(num_unique_icd9_procedure, 0)            AS num_unique_inpatient_proc,
    COALESCE(num_unique_hcpcs, 0)                     AS num_unique_inpatient_hcpcs,

    -- Outpatient
    COALESCE(outpatient_num_claims, 0)                AS outpatient_num_claims,
    COALESCE(outpatient_total_payment, 0)             AS outpatient_total_payment,
    COALESCE(outpatient_primary_payer_amt, 0)         AS outpatient_primary_payer_amt,
    COALESCE(outpatient_total_deductible, 0)          AS outpatient_total_deductible,
    COALESCE(outpatient_total_coinsurance, 0)         AS outpatient_total_coinsurance,
    COALESCE(outpatient_total_blood_deductible, 0)    AS outpatient_total_blood_deductible,
    COALESCE(num_unique_providers_1, 0)               AS num_unique_outpatient_providers,
    COALESCE(num_unique_attending_physicians_1, 0)    AS num_unique_outpatient_attending,
    COALESCE(num_unique_operating_physicians_1, 0)    AS num_unique_outpatient_operating,
    COALESCE(num_unique_other_physicians_1, 0)        AS num_unique_outpatient_other,
    COALESCE(num_unique_icd9_diagnosis_1, 0)          AS num_unique_outpatient_dx,
    COALESCE(num_unique_admitting_diagnosis, 0)       AS num_unique_outpatient_admit_dx,
    COALESCE(num_unique_icd9_procedure_1, 0)          AS num_unique_outpatient_proc,
    COALESCE(num_unique_hcpcs_1, 0)                   AS num_unique_outpatient_hcpcs,

    -- Carrier
    COALESCE(carrier_num_claims, 0)                   AS carrier_num_claims,
    COALESCE(carrier_num_lines, 0)                    AS carrier_num_lines,
    COALESCE(carrier_total_payment, 0)                AS carrier_total_payment,
    COALESCE(carrier_total_deductible, 0)             AS carrier_total_deductible,
    COALESCE(carrier_total_coinsurance, 0)            AS carrier_total_coinsurance,
    COALESCE(carrier_num_unique_hcpcs, 0)             AS carrier_num_unique_hcpcs,
    COALESCE(carrier_num_unique_service_types, 0)     AS carrier_num_unique_service_types,

    -- PDE (Drug Events)
    COALESCE(pde_num_prescriptions, 0)                AS pde_num_prescriptions,
    COALESCE(pde_num_unique_drugs, 0)                 AS pde_num_unique_drugs,
    COALESCE(pde_total_quantity, 0)                   AS pde_total_quantity,
    COALESCE(pde_avg_quantity, 0)                     AS pde_avg_quantity,
    COALESCE(pde_total_days_supply, 0)                AS pde_total_days_supply,
    COALESCE(pde_avg_days_supply, 0)                  AS pde_avg_days_supply,
    COALESCE(pde_total_patient_payment, 0)            AS pde_total_patient_payment,
    COALESCE(pde_total_drug_cost, 0)                  AS pde_total_drug_cost,
    COALESCE(pde_avg_drug_cost, 0)                    AS pde_avg_drug_cost,

    -- Derived total spending across all sources
    (COALESCE(inpatient_total_payment,0) 
     + COALESCE(outpatient_total_payment,0) 
     + COALESCE(carrier_total_payment,0) 
     + COALESCE(pde_total_drug_cost,0))               AS total_spending

FROM patient_master;
"""

conn.execute(query)
print("✅ patient_master_clean created successfully")


✅ patient_master_clean created successfully


In [163]:
df = conn.execute("select * from patient_master_clean").df()
df.count()

DESYNPUF_ID                          116352
sex                                  116352
birth_date                           116352
race                                 116352
death_date                             5461
chronic_count_2008                   116352
year                                 116352
rn                                   116352
inpatient_num_claims                 116352
inpatient_total_payment              116352
inpatient_primary_payer_amt          116352
inpatient_total_deductible           116352
inpatient_total_coinsurance          116352
inpatient_total_blood_deductible     116352
inpatient_total_util_days            116352
num_unique_inpatient_providers       116352
num_unique_inpatient_attending       116352
num_unique_inpatient_operating       116352
num_unique_inpatient_other           116352
num_unique_drg                       116352
num_unique_inpatient_dx              116352
num_unique_inpatient_proc            116352
num_unique_inpatient_hcpcs      

In [164]:
# Make sure patient_master_clean is created first
# Export to Parquet for ML use
ml_feature_path = "../features/ml_features.parquet"

# Use DuckDB's COPY TO Parquet
conn.execute(f"""
COPY patient_master_clean
TO '{ml_feature_path}'
(FORMAT PARQUET, COMPRESSION ZSTD);
""")

print(f"✅ ML feature set exported to {ml_feature_path}")


✅ ML feature set exported to ../features/ml_features.parquet
